# Subtask 1

In [94]:
def allKmerLexicographically(k, s):
    '''
    s : Dna String
    k : length of k-mer
    '''
    
    l = len(s)

    k_mer_list = []

    for i in range(l + 1 - k):
        k_mer_list.append(s[i:i+k])
    k_mer_list.sort()
    
    return k_mer_list


k = int(input())
s = input()

k_mer_list = allKmerLexicographically(k, s)

for i in k_mer_list:
    print(i)

5
CAAAGTCGGTA
AAAGT
AAGTC
AGTCG
CAAAG
CGGTA
GTCGG
TCGGT


# Subtask 2

In [95]:
def stringFromGenomePath(genome_path):
    '''
    genome_path : DNA list
    '''
    
    s = genome_path[0][:-1]
    for i in genome_path:
        s += i[-1]
    return s

genome_path = ["ACCGA",
               "CCGAA",
               "CGAAG",
               "GAAGC",
               "AAGCT"]
print(stringFromGenomePath(genome_path))


ACCGAAGCT


# Subtask 3

In [96]:
def overlapGraph(pattern_list):
    '''
    pattern_list : DNA list
    '''
    
    adjacency_list = dict()
    
    k = len(pattern_list[0])
    
    for i in pattern_list:
        suf_i = i[1:k]
        
        for j in pattern_list:
            if j != i:
                pre_j = j[0:k-1]
                if suf_i == pre_j:
                    if i in adjacency_list and not j in adjacency_list[i]:
                        adjacency_list[i].append(j)
                    else:
                        adjacency_list[i] = [j]
    return adjacency_list

pattern_list = ["ATGCG",
                "GCATG",
                "CATGC",
                "AGGCA",
                "GGCAT"]

adjacency_list_graph = overlapGraph(pattern_list)

for i in sorted(adjacency_list_graph):
    all_prefix = ""
    for j in adjacency_list_graph[i]:
        if len(all_prefix):
            all_prefix += ("," + str(j))
        else:
            all_prefix += (str(j))

    print(str(i)+ " -> " +str(all_prefix))

AGGCA -> GGCAT
CATGC -> ATGCG
GCATG -> CATGC
GGCAT -> GCATG


# Subtask 4

In [97]:
def deBruijnGraphFromString(k, s):
    '''
    k : length of k-mer 
    s : DNA String
    '''
    
    pattern_list = allKmerLexicographically(k-1, s)
    adjacency_list_graph = overlapGraph(pattern_list)
    return adjacency_list_graph
    
k = 4
s = "AAGATTCTCTAC"

adjacency_list_graph = deBruijnGraphFromString(k, s)

for i in sorted(adjacency_list_graph):
    all_prefix = ""
    for j in adjacency_list_graph[i]:
        if len(all_prefix):
            all_prefix += ("," + str(j))
        else:
            all_prefix += (str(j))

    print(str(i)+ " -> " +str(all_prefix))


AAG -> AGA
AGA -> GAT
ATT -> TTC
CTA -> TAC
CTC -> TCT
GAT -> ATT
TCT -> CTA,CTC
TTC -> TCT


# Subtask 5

In [104]:
def deBruijnGraphFromKmerList(k_mer_list):
    adjacency_list = dict()
    
    k = len(k_mer_list[0])
    
    for i in k_mer_list:
        suf_i = i[1:k]
        pre_i = i[0:k-1]

        if pre_i in adjacency_list:
            adjacency_list[pre_i].append(suf_i)
        else:
            adjacency_list[pre_i] = [suf_i]
    return adjacency_list


k_mer_list = ["GAGG",
              "CAGG",
              "GGGG",
              "GGGA",
              "CAGG",
              "AGGG",
              "GGAG"]

adjacency_list_graph = deBruijnGraphFromKmerList(k_mer_list)

for i in sorted(adjacency_list_graph):
    all_prefix = ""
    for j in adjacency_list_graph[i]:
        if len(all_prefix):
            all_prefix += ("," + str(j))
        else:
            all_prefix += (str(j))

    print(str(i)+ " -> " +str(all_prefix))

AGG -> GGG
CAG -> AGG,AGG
GAG -> AGG
GGA -> GAG
GGG -> GGG,GGA


# Subtask 6

In [93]:
import operator

def getProfileWithPseudocount(list_s):
    '''
    list_s : DNA list
    '''
    
    ans = [[],[],[],[]]
    score = 0
    t = len(list_s)
    k = len(list_s[0])
    
    for i in range(k):
        #print(i)
        probability = {'A':0.0, 'C':0.0, 'G':0.0, 'T':0.0}


        for l in list_s:
            if l[i] == 'A':
                probability['A'] += 1.0
            elif l[i] == 'C':
                probability['C'] += 1.0
            elif l[i] == 'G':
                probability['G'] += 1.0
            elif l[i] == 'T':
                probability['T'] += 1.0
        
        probability['A'] += 1.0
        probability['C'] += 1.0
        probability['G'] += 1.0
        probability['T'] += 1.0

        for j in probability.keys():
            probability[j] /= (float(t) + 4.0)

        ans[0].append(probability['A'])
        ans[1].append(probability['C'])
        ans[2].append(probability['G'])
        ans[3].append(probability['T'])
        
    return ans


def mostProbableMotif(s, k, probabilities):
    '''
    s : String from which to get the bestMotif
    k : length of k-mer
    probabilities : profile
    '''
    
    max_prob = 0.0
    profile_most = s[0: k]
    
    for i in range(len(s) - k + 1):
        sub = s[i: i + k]
        prob = 1.0
        for j in range(k):
            if sub[j] == 'A':
                prob *= probabilities[0][j]
            elif sub[j] == 'C':
                prob *= probabilities[1][j]
            elif sub[j] == 'G':
                prob *= probabilities[2][j]
            elif sub[j] == 'T':
                prob *= probabilities[3][j]

        if prob > max_prob:
            max_prob = prob
            profile_most = sub

    return profile_most


def scoreConsensus(list_s):
    '''
    list_s : DNA list
    '''
    
    t = len(list_s)
    k = len(list_s[0])

    consensus = []
    score = 0

    for i in range(k):
        #print(i)
        frequency = dict()
        max_freq = 0

        for s in list_s:
            if not s[i] in frequency:
                frequency[s[i]] = 1
            else:
                frequency[s[i]] += 1
            if frequency[s[i]] > max_freq:
                max_freq = frequency[s[i]]
                max_freq_char = s[i]

        #print(frequency)
        consensus.append(max_freq_char)
        score += (t - max_freq)

    ans = ''.join(consensus)
    return (score, ans)


def greedyMotifSearchWithPseudocount(k, list_s):
    '''
    k : length of k-mer
    list_s : DNA list
    '''
    
    t = len(list_s)
    lengthOfDna1 = len(list_s[0])
    bestMotifList = []

    for i in range(t):
        bestMotifList.append(list_s[i][0:k])

    bestScore, _ = scoreConsensus(bestMotifList)

    for i in range(lengthOfDna1 + 1 - k):
        sub = list_s[0][i:i + k]
        motif_list = []
        motif_list.append(sub)
        profile = getProfileWithPseudocount(motif_list)
        for j in range(t):
            if(j != 0):
                motif_list.append(mostProbableMotif(list_s[j], k, profile))
                profile = getProfileWithPseudocount(motif_list)
        score, _ = scoreConsensus(motif_list)
        #print(motif_list, score)
        if score < bestScore:
            bestScore = score
            bestMotifList = motif_list

    return bestMotifList


k = 3
list_s = ["GGCGTTCAGGCA",
     "AAGAATCAGTCA",
     "CAAGGAGTTCGC",
     "CACGTCAATCAC",
     "CAATAATATTCG"]

bestMotifList = greedyMotifSearchWithPseudocount(k, list_s)
for i in bestMotifList:
    print(i)

TTC
ATC
TTC
ATC
TTC


# Subtask 7

In [103]:
import random
import operator


def randomKmer(s, k):
    '''
    s : String from which to get the Kmer
    k : length of k-mer
    '''
    
    i = random.randint(0, len(s) - k)
    return s[i: i+k]


def randomMotifs(list_s, k):
    '''
    list_s : DNA list
    k : length of k-mer
    '''
    
    randomMotifList = []
    
    for s in list_s:
        randomMotifList.append(randomKmer(s, k))
        
    return randomMotifList


def getProfileWithPseudocount(list_s):
    '''
    list_s : DNA list
    '''
    
    ans = [[],[],[],[]]
    score = 0
    t = len(list_s)
    k = len(list_s[0])
    
    for i in range(k):
        #print(i)
        probability = {'A':0.0, 'C':0.0, 'G':0.0, 'T':0.0}


        for l in list_s:
            if l[i] == 'A':
                probability['A'] += 1.0
            elif l[i] == 'C':
                probability['C'] += 1.0
            elif l[i] == 'G':
                probability['G'] += 1.0
            elif l[i] == 'T':
                probability['T'] += 1.0
        
        probability['A'] += 1.0
        probability['C'] += 1.0
        probability['G'] += 1.0
        probability['T'] += 1.0

        for j in probability.keys():
            probability[j] /= (float(t) + 4.0)

        ans[0].append(probability['A'])
        ans[1].append(probability['C'])
        ans[2].append(probability['G'])
        ans[3].append(probability['T'])
        
    return ans


def mostProbableMotif(s, k, probabilities):
    '''
    s : String from which to get the bestMotif
    k : length of k-mer
    probabilities : profile
    '''
    
    max_prob = 0.0
    profile_most = s[0: k]
    
    for i in range(len(s) - k + 1):
        sub = s[i: i + k]
        prob = 1.0
        for j in range(k):
            if sub[j] == 'A':
                prob *= probabilities[0][j]
            elif sub[j] == 'C':
                prob *= probabilities[1][j]
            elif sub[j] == 'G':
                prob *= probabilities[2][j]
            elif sub[j] == 'T':
                prob *= probabilities[3][j]

        if prob > max_prob:
            max_prob = prob
            profile_most = sub

    return profile_most


def scoreConsensus(list_s):
    '''
    list_s : DNA list
    '''
    
    t = len(list_s)
    k = len(list_s[0])

    consensus = []
    score = 0

    for i in range(k):
        #print(i)
        frequency = dict()
        max_freq = 0

        for s in list_s:
            if not s[i] in frequency:
                frequency[s[i]] = 1
            else:
                frequency[s[i]] += 1
            if frequency[s[i]] > max_freq:
                max_freq = frequency[s[i]]
                max_freq_char = s[i]

        #print(frequency)
        consensus.append(max_freq_char)
        score += (t - max_freq)

    ans = ''.join(consensus)
    return (score, ans)


def repeatedRandomizedMotifSearch(k, t, list_s):
    '''
    k : length of k-mer
    t : total number of DNA in DNA list
    list_s : DNA list
    '''
    
    bestScore = float('inf')
    bestMotifList=[]
    i = 0
    
    while True:
        motifs = randomizedMotifSearch(k, t, list_s)
        score, _ = scoreConsensus(motifs)
        
        if score < bestScore:
            bestScore = score
            bestMotifList = motifs
            i = 0
        else:
            i += 1
            
        if i > 1000:
            break;
            
    return bestMotifList
        
    
def randomizedMotifSearch(k, t, list_s):
    '''
    k : length of k-mer
    t : total number of DNA in DNA list
    list_s : DNA list
    '''
    
    lengthOfDna1 = len(list_s[0])
    bestMotifList = randomMotifs(list_s, k)

    bestScore, _ = scoreConsensus(bestMotifList)

    while True:
        
        profile = getProfileWithPseudocount(bestMotifList)
        motif_list = [mostProbableMotif(s, k, profile) for s in list_s]
        score, _ = scoreConsensus(motif_list)
        
        if score < bestScore:
            bestScore = score
            bestMotifList = motif_list
        else:
            return bestMotifList


k = 8 
t = 5
list_s = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
          "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
          "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
          "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
          "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

bestMotifList = repeatedRandomizedMotifSearch(k, t, list_s)
for i in bestMotifList:
    print(i)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG
